In [2]:
import logging
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [8]:
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }
#URL = "https://www.myg.in/?match=all&subcats=Y&pcode_from_q=Y&pshort=Y&pfull=Y&pname=Y&pkeywords=Y&search_performed=Y&q=phone+&dispatch=products.search&security_hash=0a4817ffa50215e80fea73a431c5e4e2"
URL = "https://www.amazon.in/hz/mobile/mission?p=Awvb5PYPCdXItKNk4syR9w%2F8ZaEI%2FEWhSLq8neDBm2ZDN%2FiUpGTmmeLEaXhYjTydaUP1RHT%2BGq3wDqgl1VsLnYaGIm60jpyMp%2FENkU67HV9BrEL%2FMuJsyHoJAVbyza4fZgpw53GCiKu"

In [25]:
def fetch_and_parse(URL:str,headers:dict):
    """Validate the URL, fetch the webpage, and parse the content using BeautifulSoup."""

    parsed_url = urlparse(URL)
    if not parsed_url.scheme:
        URL = f"http://{URL}"

    try:
        response = requests.get(URL, headers=headers)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, "html.parser")
        print(soup)
    except:
        pass 

In [42]:
import logging
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from utils import *



logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [48]:
from bs4 import BeautifulSoup
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def extract_text_from_html_file(file_path: str) -> str:
    """Extract text from paragraph (<p>) tags in an HTML file.

    Args:
        file_path (str): The path to the HTML file.

    Returns:
        str: The cleaned text extracted from the HTML file.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        soup = BeautifulSoup(html_content, "html.parser")

        for unwanted in soup(["script", "style"]):
            unwanted.extract()


        paragraphs = soup.find_all('p')
        if paragraphs:
            text_content = ' '.join(para.get_text(separator=' ', strip=True) for para in paragraphs)
            logger.info("Text successfully extracted from paragraph tags.")
            return text_content
        else:
            logger.warning("No paragraph tags found in the HTML file.")
            return "No content found."
    
    except FileNotFoundError:
        logger.error(f"File not found: {file_path}")
        return "File not found."
    
    except Exception as e:
        logger.error(f"An error occurred while processing the file: {e}")
        return "An error occurred."

if __name__ == "__main__":
    file_path = "htmls.html"  
    extracted_text = extract_text_from_html_file(file_path)
    print(extracted_text)  


No content found.


In [49]:
%pwd 

'c:\\Users\\Asus\\vs_code\\Internship\\Rough'

In [50]:
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

In [52]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [51]:
import logging
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse



logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class WebPageScraper:
    def __init__(self, url: str, headers: dict, save_path: str = "output_context.txt"):
        """
        Initialize the WebPageScraper with the given URL, headers, and output file path.

        Args:
            url (str)       : The URL of the webpage to scrape.
            headers (dict)  : The headers to include in the HTTP request.
            save_path (str) : The path to the output file where extracted text will be saved.
        """
        self.URL        = url
        self.soup       = None
        self.headers    = headers 
        self.save_path  = save_path

    def fetch_and_parse(self):
        """Validate the URL, fetch the webpage, and parse the content using BeautifulSoup."""
    
        parsed_url = urlparse(self.URL)
        if not parsed_url.scheme:
            self.URL = f"http://{self.URL}"

        try:
            response = requests.get(self.URL, headers=self.headers)
            response.raise_for_status()  # Check for HTTP errors
            self.soup = BeautifulSoup(response.text, "html.parser")
            logger.info(f"Successfully fetched and parsed: {self.URL}")

        except requests.exceptions.MissingSchema:
            logger.error(f"Invalid URL: {self.URL} - Missing schema.")
            raise ValueError(f"Invalid URL: {self.URL} - Please provide a valid URL.")
        
        except requests.exceptions.ConnectionError:
            logger.error(f"Connection error while trying to reach {self.URL}.")
            raise ConnectionError(f"Failed to connect to {self.URL}.")
        
        except requests.exceptions.Timeout:
            logger.error(f"Timeout error while trying to reach {self.URL}.")
            raise TimeoutError(f"Request to {self.URL} timed out.")
        
        except requests.exceptions.HTTPError as err:
            logger.error(f"HTTP error occurred: {err}")
            raise RuntimeError(f"HTTP error occurred: {err}")
        
        except Exception as e:
            logger.error(f"An unexpected error occurred: {e}")
            raise RuntimeError(f"An unexpected error occurred: {e}")

    def clean_and_extract_text(self) -> str:
        """Remove unwanted tags and extract text from paragraph (<p>) tags.

        Returns:
            str: The cleaned text extracted from the webpage.
        """
        if self.soup:
            for unwanted in self.soup(["script", "style", "a"]):
                unwanted.extract()
            paragraphs = self.soup.find_all('p')
            if paragraphs:
                text_content = ' '.join(para.get_text(separator=' ', strip=True) for para in paragraphs)
                logger.info("Text successfully extracted from paragraph tags.")
                return text_content
            else:
                logger.warning("No paragraph tags found in the response.")
                return "No content found."
        logger.warning("No soup object available for text extraction.")
        return None

    def save_text_to_file(self, text: str):
        """Save the extracted text to a specified output file.

        Args:
            text (str): The text content to save.
        """
        with open(self.save_path, 'w', encoding='utf-8') as file:
            file.write(text)
            logger.info(f"Extracted text saved to {self.save_path}")

# Example usage
if __name__ == "__main__":
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }

    #URL = "https://www.myg.in/?match=all&subcats=Y&pcode_from_q=Y&pshort=Y&pfull=Y&pname=Y&pkeywords=Y&search_performed=Y&q=phone+&dispatch=products.search&security_hash=0a4817ffa50215e80fea73a431c5e4e2"
    URL = "https://www.amazon.in/hz/mobile/mission?p=Awvb5PYPCdXItKNk4syR9w%2F8ZaEI%2FEWhSLq8neDBm2ZDN%2FiUpGTmmeLEaXhYjTydaUP1RHT%2BGq3wDqgl1VsLnYaGIm60jpyMp%2FENkU67HV9BrEL%2FMuJsyHoJAVbyza4fZgpw53GCiKuku%2Fy6GnDuUrki5PD6ijLBckEJuQwlkV8YnUajF8b2lBzL6SGo3urB4TT0o%2FR62IB%2BV2LZ0IeGB44TjQMhlNRmqGLs%2F2BBqbtKgkq5wujUsFBrDZ0FiTFqh%2Bby52Ea62F3hAMzSd%2BOTPSnmrUyiMI74568Ei%2Bew%2Bss36R%2FWtp6EjUVrIQjRTr7ofP3t9V4LiQBhk%2FtgSCTT7q71PLF%2F2t8%2FvM%2Fbi%2B%2FQp3Iaq7f5dfeJuXVim8CRIFFRTbNoNTKrlb1XZ%2FWlRnIDe5gYKbo0TfEJ1B%2F57iLhT8%3D&ref_=ci_mcx_mi&pf_rd_r=5NDCMJ3514XZZF96VSK8&pf_rd_p=652a835d-9e23-4efd-9931-74188247a57a&pd_rd_r=744eadd7-6515-444f-bbec-acd0af863342&pd_rd_w=E6X0m&pd_rd_wg=SBqVS"
    URL = "https://www.amazon.in/Samsung-Galaxy-Snapdragon-Phantom-Storage/dp/B0BTYX74HZ/ref=pd_ci_mcx_mh_mcx_views_0?pd_rd_w=hfy8A&content-id=amzn1.sym.fa0aca50-60f7-47ca-a90e-c40e2f4b46a9%3Aamzn1.symc.ca948091-a64d-450e-86d7-c161ca33337b&pf_rd_p=fa0aca50-60f7-47ca-a90e-c40e2f4b46a9&pf_rd_r=MHZ5Y02EQCB7YRH7TN1Q&pd_rd_wg=ZBuwO&pd_rd_r=d48e3b16-3e62-408a-b42a-17490e99d283&pd_rd_i=B0BTYX74HZ"    
    scraper = WebPageScraper(url=URL, headers=headers)
    
    try:
        scraper.fetch_and_parse()
        extracted_text = scraper.clean_and_extract_text()
        if extracted_text:
            scraper.save_text_to_file(extracted_text)
    except Exception as e:
        logger.error(f"Error during scraping: {e}")


INFO:__main__:Successfully fetched and parsed: https://www.amazon.in/Samsung-Galaxy-Snapdragon-Phantom-Storage/dp/B0BTYX74HZ/ref=pd_ci_mcx_mh_mcx_views_0?pd_rd_w=hfy8A&content-id=amzn1.sym.fa0aca50-60f7-47ca-a90e-c40e2f4b46a9%3Aamzn1.symc.ca948091-a64d-450e-86d7-c161ca33337b&pf_rd_p=fa0aca50-60f7-47ca-a90e-c40e2f4b46a9&pf_rd_r=MHZ5Y02EQCB7YRH7TN1Q&pd_rd_wg=ZBuwO&pd_rd_r=d48e3b16-3e62-408a-b42a-17490e99d283&pd_rd_i=B0BTYX74HZ
INFO:__main__:Text successfully extracted from paragraph tags.
INFO:__main__:Extracted text saved to output_context.txt


In [40]:
extracted_text

"Top Brand Samsung 89% positive ratings from 100K+ customers 100K+ recent orders from this brand 11+ years on Amazon Give yourself a smartphone that recognises your emotions and responds appropriately. The Samsung Galaxy S23 5G's enhanced AI and Nightography feature produces low-light photos and videos that are vivid and colourful from dusk to dawn and back again. The Snapdragon processor in this phone also offers quick video streaming and gaming. Additionally, adaptive 120 Hz makes scrolling fluid, and Eye Comfort Shield guards against eye fatigue even while looking in low light. Samsung Galaxy S23 5G Snapdragon (Phantom Black, 8GB, 128GB Storage) Share: Found a lower price? Let us know. Fields with an asterisk * are required"